# Model with element sequence recognition - 6 characters - value encoded prediction

Builds on `RNN-Morse-chars-single-ddp06` with value encoded prediction. The sequence of elements is ternary encoded in reverse order relative to their position. The example below is for 3 elements (14 characters) encoding on ternary values. The rationale is that encoded characters with maximum similarity will have the minimal distance between them. For example:

  - S `...`  is 13 and U `..-` is 14. 
  - I `..` is 12, A `.-` is 15, N `-.` is 21 thus relative position matters the maximum similarity being at the start of sequence.
  - Conversely K `-.-` is 23 and R `.-.` is 16, 
  - E `.` is 9, T `-` is 18. 
  
Extrema are E (9) and O (26). The shift by 9 can be corrected by subtracting 8 to positive values. Generally: 3^(max_ele-1) - 1

| element   | pos0              | pos1             | pos2             |
| --------- | ----------------- | ---------------- | ---------------- |
| n/a (0)   | +0                | +0               | +0               |
| dit (1)   | +3^2 (9)          | +3^1 (3)         | +3^0   (1)       |
| dah (2)   | +2&times;3^2 (18) | +2&times;3^1 (6) | +2&times;3^0 (2) |


## Create string

Each character in the alphabet should happen a large enough number of times. As a rule of thumb we will take some multiple of the number of characters in the alphabet. If the multiplier is large enough the probability of each character appearance will be even over the alphabet. 

Seems to get better results looking at the gated graphs but procedural decision has to be tuned.

In [ ]:
import MorseGen

morse_gen = MorseGen.Morse()
alphabet = morse_gen.alphabet6
print(132/len(alphabet))

morsestr = MorseGen.get_morse_str(nchars=132*3, nwords=27*3, chars=alphabet)
print(alphabet)
print(len(morsestr), morsestr)

## Generate dataframe and extract envelope

In [ ]:
Fs = 8000
samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
n_prev = int((samples_per_dit/128)*12*2) + 1
print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
label_df = morse_gen.encode_df_decim_val(morsestr, samples_per_dit, 128, alphabet)
env = label_df['env'].to_numpy()
print(type(env), len(env))

In [ ]:
import numpy as np

def get_new_data(morse_gen, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
    if not phrase:
        phrase = MorseGen.get_morse_str(nchars=nchars, nwords=nwords, chars=alphabet)
    print(len(phrase), phrase)
    Fs = 8000
    samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
    n_prev = int((samples_per_dit/128)*19) + 1 # number of samples to look back is slightly more than a "O" a word space (3*4+7=19)
    #n_prev = int((samples_per_dit/128)*27) + 1 # number of samples to look back is slightly more than a "0" a word space (5*4+7=27)
    print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/128:.2f}. Look back is {n_prev}.')
    label_df = morse_gen.encode_df_decim_val(phrase, samples_per_dit, 128, alphabet)
    # extract the envelope
    envelope = label_df['env'].to_numpy()
    # remove the envelope
    label_df.drop(columns=['env'], inplace=True)
    SNR_linear = 10.0**(SNR_dB/10.0)
    SNR_linear *= 256 # Apply original FFT
    print(f'Resulting SNR for original {SNR_dB} dB is {(10.0 * np.log10(SNR_linear)):.2f} dB')
    t = np.linspace(0, len(envelope)-1, len(envelope))
    power = np.sum(envelope**2)/len(envelope)
    noise_power = power/SNR_linear
    noise = np.sqrt(noise_power)*np.random.normal(0, 1, len(envelope))
    # noise = butter_lowpass_filter(raw_noise, 0.9, 3) # Noise is also filtered in the original setup from audio. This empirically simulates it
    signal = (envelope + noise)**2
    signal[signal > 1.0] = 1.0 # a bit crap ...
    return envelope, signal, label_df, n_prev, morse_gen.max_ele(alphabet)

Try it...

In [ ]:
import matplotlib.pyplot as plt 

envelope, signal, label_df, n_prev, max_ele = get_new_data(morse_gen, SNR_dB=-17, phrase=morsestr, alphabet=alphabet)

# Show
print(n_prev)
print(type(signal), signal.shape)
print(type(label_df), label_df.shape)
    
x0 = 0
x1 = 1500

plt.figure(figsize=(50,20))
plt.plot(signal[x0:x1]*0.7, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
plt.plot(label_df[x0:x1].ele*0.9 + 1.0, label='ele')
plt.plot(label_df[x0:x1].chr*0.9 + 1.0, label='chr', color="orange")
plt.plot(label_df[x0:x1].wrd*0.9 + 1.0, label='wrd')
plt.plot(label_df[x0:x1].val + 2.0, label='val')
plt.title("signal and labels")
plt.legend(loc=2)
plt.grid()

## Create data loader
### Define dataset

In [ ]:
import torch

class MorsekeyingDataset(torch.utils.data.Dataset):
    def __init__(self, morse_gen, device, SNR_dB=-23, nchars=132, nwords=27, phrase=None, alphabet="ABC"):
        self.envelope, self.signal, self.label_df0, self.seq_len, self.max_ele = get_new_data(morse_gen, SNR_dB=SNR_dB, phrase=phrase, alphabet=alphabet)
        self.label_df = self.label_df0
        self.X = torch.FloatTensor(self.signal).to(device)
        self.y = torch.FloatTensor(self.label_df.values).to(device)
        
    def __len__(self):
        return self.X.__len__() - self.seq_len

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len])
    
    def get_envelope(self):
        return self.envelope
    
    def get_signal(self):
        return self.signal
    
    def get_X(self):
        return self.X
    
    def get_labels(self):
        return self.label_df
    
    def get_labels0(self):
        return self.label_df0
    
    def get_seq_len(self):
        return self.seq_len()
    
    def max_ele(self):
        return self.max_ele

### Define keying data loader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_chr_dataset = MorsekeyingDataset(morse_gen, device, -20, 132*5, 27*5, morsestr, alphabet)
train_chr_loader = torch.utils.data.DataLoader(train_chr_dataset, batch_size=1, shuffle=False) # Batch size must be 1

In [ ]:
signal = train_chr_dataset.get_signal()
envelope = train_chr_dataset.get_envelope()
label_df = train_chr_dataset.get_labels()
label_df0 = train_chr_dataset.get_labels0()

print(type(signal), signal.shape)
print(type(label_df), label_df.shape)

x0 = 0
x1 = 1500

plt.figure(figsize=(50,4))
plt.plot(signal[x0:x1]*0.8, label="sig", color="cornflowerblue")
plt.plot(envelope[x0:x1]*0.9, label='env', color="orange")
plt.plot(label_df[x0:x1].ele*0.9 + 1.0, label='ele', color="orange")
plt.plot(label_df[x0:x1].chr*0.9 + 1.0, label='chr', color="green")
plt.plot(label_df[x0:x1].wrd*0.9 + 1.0, label='wrd', color="red")
plt.plot(label_df[x0:x1].val + 2.0, label='val', color="darkturquoise")
plt.title("keying - signal and labels")
plt.legend(loc=2)
plt.grid()

## Create model classes

In [ ]:
import torch
import torch.nn as nn

class MorseLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        )        
    
class MorseBatchedLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))
    
    def _minmax(self, x):
        x -= x.min(0)[0]
        x /= x.max(0)[0]
        
    def _hardmax(self, x):
        x /= x.sum()
        
    def _sqmax(self, x):
        x = x**2
        x /= x.sum()
        
    def forward(self, input_seq):
        #print(len(input_seq), input_seq.shape, input_seq.view(-1, 1, 1).shape)
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(-1, 1, self.input_size), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        self._sqmax(predictions[-1])
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        )     
    
class MorseLSTM2(nn.Module):
    """
    LSTM stack
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6, dropout=0.2):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=2, dropout=dropout)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(2, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(2, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(2, 1, self.hidden_layer_size).to(device),
            torch.zeros(2, 1, self.hidden_layer_size).to(device)
        )        
        
class MorseNoHLSTM(nn.Module):
    """
    Do not keep hidden cell
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        h0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        c0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1), (h0, c0))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
class MorseBiLSTM(nn.Module):
    """
    Attempt Bidirectional LSTM: does not work
    """
    def __init__(self, device, input_size=1, hidden_size=12, num_layers=1, num_classes=6):
        super(MorseEnvBiLSTM, self).__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x.view(len(x), 1, -1), (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out[-1]    

Create the keying model instance and print the details

In [ ]:
morse_chr_model = MorseBatchedLSTM(device, hidden_layer_size=len(alphabet), output_size=4).to(device) # This is the only way to get things work properly with device
morse_chr_loss_function = nn.MSELoss()
morse_chr_optimizer = torch.optim.Adam(morse_chr_model.parameters(), lr=0.001)

print(morse_chr_model)
print(morse_chr_model.device)

In [ ]:
# Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu
for m in morse_chr_model.parameters():
    print(m.shape, m.device)
X_t = torch.rand(n_prev)
X_t = X_t.cuda()
print("Input shape", X_t.shape, X_t.view(-1, 1, 1).shape)
print(X_t)
morse_chr_model(X_t)

In [ ]:
import torchinfo
torchinfo.summary(morse_chr_model)

## Train model

In [ ]:
it = iter(train_chr_loader)
X, y = next(it)
print(X.reshape(n_prev,1).shape, X[0].shape, y[0].shape)
print(X[0], y[0])
X, y = next(it)
print(X[0], y[0])

In [ ]:
%%time
from tqdm.notebook import tqdm

epochs = 4
morse_chr_model.train()

for i in range(epochs):
    train_losses = []
    loop = tqdm(enumerate(train_chr_loader), total=len(train_chr_loader), leave=True)
    for j, train in loop:
        X_train = train[0][0]
        y_train = train[1][0]
        morse_chr_optimizer.zero_grad()
        if morse_chr_model.__class__.__name__ in ["MorseLSTM", "MorseLSTM2", "MorseBatchedLSTM", "MorseBatchedLSTM2"]:
            morse_chr_model.zero_hidden_cell() # this model needs to reset the hidden cell
        y_pred = morse_chr_model(X_train)
        single_loss = morse_chr_loss_function(y_pred, y_train)
        single_loss.backward()
        morse_chr_optimizer.step()
        train_losses.append(single_loss.item())
        # update progress bar
        if j % 1000 == 0:
            loop.set_description(f"Epoch [{i+1}/{epochs}]")
            loop.set_postfix(loss=np.mean(train_losses))

print(f'final: {i+1:3} epochs loss: {np.mean(train_losses):6.4f}')

In [ ]:
save_model = True
if save_model: 
    torch.save(morse_chr_model.state_dict(), 'models/morse_a06_model')
else:
    morse_chr_model.load_state_dict(torch.load('models/morse_singlemm_model', map_location=device))

In [ ]:
%%time
p_char_train = torch.empty(1,4).to(device)
morse_chr_model.eval()

loop = tqdm(enumerate(train_chr_loader), total=len(train_chr_loader))
for j, train in loop:
    with torch.no_grad():
        X_chr = train[0][0]
        pred_val = morse_chr_model(X_chr)
        p_char_train = torch.cat([p_char_train, pred_val.reshape(1,4)])        

In [ ]:
p_char_train = p_char_train[1:] # Remove garbge
print(p_char_train.shape) # t -> chars(t)

### Post process
  
  - Move to CPU to ger chars(time)
  - Transpose to get times(char)

In [ ]:
p_char_train_c = p_char_train.cpu() # t -> chars(t) on CPU
p_char_train_t = torch.transpose(p_char_train_c, 0, 1).cpu() # c -> times(c) on CPU
print(p_char_train_c.shape, p_char_train_t.shape)

In [ ]:
X_train_chr = train_chr_dataset.X.cpu()
label_df_chr = train_chr_dataset.get_labels()

l_alpha = label_df_chr[n_prev:].reset_index(drop=True)
plt.figure(figsize=(50,4))
plt.plot(l_alpha[x0:x1]["chr"]*4, label="ychr", alpha=0.2, color="black")
plt.plot(X_train_chr[x0+n_prev:x1+n_prev]*0.9, label='sig')
plt.plot(p_char_train_t[0][x0:x1]*0.9 + 1.0, label='e', color="orange")
plt.plot(p_char_train_t[1][x0:x1]*0.9 + 2.0, label='c', color="green")
plt.plot(p_char_train_t[2][x0:x1]*0.9 + 2.0, label='w', color="red")
plt.plot(p_char_train_t[3][x0:x1] + 3.0, label='v', color="purple")
plt.title("predictions")
plt.legend(loc=2)
plt.grid()

## Test

### Test dataset and data loader

In [ ]:
teststr = "AA MA MINETTE TETA ET MA MAMA MINE AA"
test_chr_dataset = MorsekeyingDataset(morse_gen, device, -17, 132*5, 27*5, teststr, alphabet)
test_chr_loader = torch.utils.data.DataLoader(test_chr_dataset, batch_size=1, shuffle=False) # Batch size must be 1

### Run the model

In [ ]:
p_chr_test = torch.empty(1,max_ele+3).to(device)
morse_chr_model.eval()

loop = tqdm(enumerate(test_chr_loader), total=len(test_chr_loader))
for j, test in loop:
    with torch.no_grad():
        X_test = test[0]
        pred_val = morse_chr_model(X_test[0])
        p_chr_test = torch.cat([p_chr_test, pred_val.reshape(1,max_ele+3)])

In [ ]:
# drop first garbage sample
p_chr_test = p_chr_test[1:]
print(p_chr_test.shape)

In [ ]:
p_chr_test_c = p_chr_test.cpu() # t -> chars(t) on CPU
p_chr_test_t = torch.transpose(p_chr_test_c, 0, 1).cpu() # c -> times(c) on CPU
print(p_chr_test_c.shape, p_chr_test_t.shape)

### Show results

In [ ]:
X_test_chr = test_chr_dataset.X.cpu()
label_df_t = test_chr_dataset.get_labels()
l_alpha_t = label_df_t[n_prev:].reset_index(drop=True)

#### Raw results

In [ ]:
plt.figure(figsize=(100,4))
plt.plot(l_alpha_t[:]["chr"]*4, label="ychr", alpha=0.2, color="black")
plt.plot(X_test_chr[n_prev:]*0.9, label='sig')
plt.plot(p_chr_test_t[0]*0.9 + 1.0, label='e', color="purple")
plt.plot(p_chr_test_t[1]*0.9 + 2.0, label='c', color="green")
plt.plot(p_chr_test_t[2]*0.9 + 2.0, label='w', color="red")
for i in range(max_ele):
    plt_a = plt.plot(p_chr_test_t[i+3]*0.9 + 3.0, label=f'e{i}')
plt.title("predictions")
plt.legend(loc=2)
plt.grid()
plt.savefig('img/predicted.png')

In [ ]:
p_chr_test_tn = p_chr_test_t.numpy()
ele_len = round(samples_per_dit / 256)
win = np.ones(ele_len)/ele_len
p_chr_test_tlp = np.apply_along_axis(lambda m: np.convolve(m, win, mode='full'), axis=1, arr=p_chr_test_tn)

plt.figure(figsize=(100,4))
plt.plot(l_alpha_t[:]["chr"]*4, label="ychr", alpha=0.2, color="black")
plt.plot(X_test_chr[n_prev:]*0.9, label='sig')
plt.plot(p_chr_test_tlp[0]*0.9 + 1.0, label='e', color="purple")
plt.plot(p_chr_test_tlp[1]*0.9 + 2.0, label='c', color="green")
plt.plot(p_chr_test_tlp[2]*0.9 + 2.0, label='w', color="red")
for i in range(max_ele):
    plt.plot(p_chr_test_tlp[i+3,:]*0.9 + 3.0, label=f'e{i}')
plt.title("predictions")
plt.legend(loc=2)
plt.grid()
plt.savefig('img/predicted.png')

#### Gated by character prediction

## Procedural decision making TBD

### take 2

In [ ]:
class MorseDecoder2:
    def __init__(self, alphabet, chr_len, wrd_len):
        self.nb_alpha = len(alphabet)
        self.alphabet = alphabet
        self.chr_len = chr_len
        self.wrd_len = wrd_len // 2
        self.threshold = 0.25
        self.chr_count = 0
        self.wrd_count = 0        
        self.prevs = [0.0 for x in range(self.nb_alpha+3)]
        self.res = ""
    
    def new_samples(self, samples):
        for i, s in enumerate(samples): # c, w, n, [alpha]
            if i > 1:
                t = s * samples[0] # gating for alpha characters
            else:
                t = s
            if i == 1: # word separator
                if t >= self.threshold and self.prevs[1] < self.threshold and self.wrd_count == 0:
                    self.wrd_count = self.wrd_len
                    self.res += " "
            elif i > 1: # characters
                if t >= self.threshold and self.prevs[i] < self.threshold and self.chr_count == 0:
                    self.chr_count = self.chr_len
                    if i > 2:
                        self.res += self.alphabet[i-3]
            self.prevs[i] = t
        if self.wrd_count > 0:
            self.wrd_count -= 1
        if self.chr_count > 0:
            self.chr_count -= 1                    

In [ ]:
chr_len = round(samples_per_dit*2 / 128)
wrd_len = round(samples_per_dit*4 / 128)
decoder = MorseDecoder2(alphabet, chr_len, wrd_len)
#p_chr_test_clp = torch.transpose(p_chr_test_tlp, 0, 1)
p_chr_test_clp = p_chr_test_tlp.transpose()
for s in p_chr_test_clp:
    decoder.new_samples(s[1:]) # c, w, n, [alpha]
print(decoder.res)

### take 1

In [ ]:
class MorseDecoder1:
    def __init__(self, alphabet, chr_len, wrd_len):
        self.nb_alpha = len(alphabet)
        self.alphabet = alphabet
        self.chr_len = chr_len
        self.wrd_len = wrd_len
        self.alpha = 0.3
        self.threshold = 0.45
        self.accum = [0.0 for x in range(self.nb_alpha+2)] 
        self.sums = [0.0 for x in range(self.nb_alpha+2)]
        self.tests = [0.0 for x in range(self.nb_alpha+2)]
        self.prevs = [0.0 for x in range(self.nb_alpha+2)]
        self.counts = [0 for x in range(self.nb_alpha+2)]
        self.res = ""

    def new_samples(self, samples):
        for i, s in enumerate(samples):
            if i > 2:
                t = s * samples[0] # gating for alpha characters
            else:
                t = s
#             t = s
            if i > 0:
                j = i-1
                t = self.alpha * t + (1 - self.alpha) * self.accum[j] # Exponential average does the low pass filtering
                self.accum[j] = t
                if t >= self.threshold and self.prevs[j] < self.threshold:
                    self.counts[j] = 0
                if t > self.threshold:
                    self.sums[j] = self.sums[j] + t
                    self.tests[j] = 0.0
                else:
                    blk_len = wrd_len if j == 0 else chr_len
                    if self.counts[j] > blk_len:
                        self.tests[j] = self.sums[j]
                        self.sums[j] = 0.0
                self.counts[j] += 1
                self.prevs[j] = t
        if np.sum(self.tests) > 0.0:
            ci = np.argmax(self.tests)
            if ci == 0:
                self.res += " "
            elif ci > 1:
                self.res += self.alphabet[ci - 2]

In [ ]:
chr_len = round(samples_per_dit*2 / 128)
wrd_len = round(samples_per_dit*4 / 128)
decoder = MorseDecoder1(alphabet, chr_len, wrd_len)
for s in p_chr_test_c:
    decoder.new_samples(s[1:]) # c, w, n, [alpha]
print(decoder.res)